In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from keras.datasets import fashion_mnist
import matplotlib.pyplot as plt
from keras import models
from keras import layers
from keras.utils import to_categorical

In [2]:
def pre_processamento():
    (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

    train_images = train_images.reshape((60000,28,28,1))
    train_images = train_images.astype('float32')/255

    test_images = test_images.reshape((10000,28,28,1))
    test_images = test_images.astype('float32')/255

    train_labels = to_categorical(train_labels)
    test_labels = to_categorical(test_labels)

    return train_images, train_labels, test_images, test_labels

In [3]:
def create_cnn_model(num_conv_layers):
    model = tf.keras.models.Sequential()

    # Camadas de convolução-pooling
    for i in range(num_conv_layers):
        model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu', input_shape=(28, 28, 1)))
        model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax')) # 10 é o número de classes

    return model


In [7]:
# Parâmetros
num_conv_layers_list = [1, 2, 3]
num_folds = 5

# Lista para armazenar as acurácias
accuracies = {}

# Pré-processamento dos dados
train_images, train_labels, test_images, test_labels = pre_processamento()


melhor = 0
num_camada = 0

# Cross-validation
for num_conv_layers in num_conv_layers_list:
    fold_accuracies = []
    for fold in range(num_folds):
        # Dividir os dados de treinamento em folds de treinamento e validação
        fold_size = len(train_images) // num_folds
        fold_start = fold * fold_size
        fold_end = (fold + 1) * fold_size

        x_val_fold = train_images[fold_start:fold_end]
        y_val_fold = train_labels[fold_start:fold_end]

        train_images_fold = np.concatenate([train_images[:fold_start], train_images[fold_end:]])
        train_labels_fold = np.concatenate([train_labels[:fold_start], train_labels[fold_end:]])

        # Criar o modelo CNN
        model = create_cnn_model(num_conv_layers)

        # Compilar e treinar o modelo
        model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
        model.fit(train_images_fold, train_labels_fold, epochs=10, batch_size=64, verbose=0)

        # Avaliar o modelo no fold de validação
        _, accuracy = model.evaluate(x_val_fold, y_val_fold, verbose=0)
        fold_accuracies.append(accuracy)

    # Calcular a acurácia média para o número atual de camadas de convolução-pooling
    avg_accuracy = np.mean(fold_accuracies)
    accuracies.update({num_conv_layers:avg_accuracy})
    if accuracies[num_conv_layers] > melhor:
        melhor = accuracies[num_conv_layers]
        num_camada = num_conv_layers

    print("Número de camadas de convolução-pooling:", num_conv_layers)
    print("Acurácia média:", avg_accuracy)

print(f"Portanto, o melhor número de camadas é {num_camada}, com a acurácia de {melhor}")

# Imprimir as acurácias para cada quantidade de camadas de convolução-pooling

# for layer in accuracies.keys():
#     print("Número de camadas de convolução-pooling:", layer)
#     print("Acurácia média:", accuracies[layer])
#     if accuracies[layer] > melhor:
#         melhor = accuracies[layer]
#         num_camada = layer
# print(f"Portanto, o melhor número de camadas é {num_camada}, com a acurácia de {melhor}")

Número de camadas de convolução-pooling: 1
Acurácia média: 0.916016674041748
Número de camadas de convolução-pooling: 2
Acurácia média: 0.9185333251953125
Número de camadas de convolução-pooling: 3
Acurácia média: 0.9064166665077209
Portanto, o melhor número de camadas é 2, com a acurácia de 0.9185333251953125
